In [1]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
from numpy import genfromtxt

/Users/andrew/workspace/endangeredanimals/analysis


/Users/andrew/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
t = pd.read_csv("labeled.csv", encoding="ISO-8859-1", index_col=0)
t.set_index('_id')
t

,_id,ltable_name,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,ltable_status,ltable_country_count,...,rtable_size,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_status,rtable_countries,rtable_country_count,rtable_tCount,label
7,1522478,caterpillar slug,Veronicellidae,NaN,South Africa (KwaZulu-Natal);,NaN,NaN,Unknown,Endangered,1,...,Extended length: up to 90 mm (2),The caterpillar slug is threatened by habitat loss and degradation as a result of ongoing urbani...,Although there are currently no conservation measures directly targeting the caterpillar slug in...,loss;environment;,,Endangered,"['India', 'Russia', 'Malaysia', 'China', 'Indonesia']",5,2,1.0
17,8689373,catalina mahogany,Rosaceae,NaN,United States (California);,NaN,NaN,NaN,Critically Endangered,1,...,Height: 3 - 7 m (2)Trunk diameter: c. 20 cm (2),Historically a major threat to the Catalina mahogany was the introduction of <strong>herbivores<...,Conservation efforts began in the 1970s with a detailed inventory of the remaining Catalina maho...,loss;invasive;,,Critically Endangered,"['Ukraine', 'Morocco', 'Russia', 'Hungary']",4,2,1.0
21,7512846,"lorenz von liburnaus woolly lemur, western avahi, western woolly lemur",Indriidae,Terrestrial,Madagascar;,\r\r\r\r\r\n The major threat is forest destruction due to annual burning that creates new ca...,"['\n This species is listed on Appendix I of CITES. ', <span lang=""EN-CA"">This species is kno...",Decreasing,Endangered,1,...,700 Ã¢?? 900 g (2),15 species of lemur have become extinct since sea-faring humans arrived on MadagascarÃ¢??s shore...,The western woolly lemur is confirmed in only two protected areas Ankarafantsika Nature Reserve ...,hunting;,protected;,Critically Endangered,['Australia'],1,1,1.0
23,1191527,"bluelegged mantella, tular golden frog, tular mantella, tulear golden frog",Mantellidae,Terrestrial; Freshwater,Madagascar;,"\r\r\r\r\r\n The main threat to this species is habitat loss due to grazing and fire, and in ...",['\n It occurs in Parque Nacional de Isalo. Trade in this species needs to be very carefully ...,Decreasing,Endangered,1,...,1 Ã¢?? 3 g (3),Several thousand blue-legged mantellas are thought to be collected every year from some regions ...,Listing on Appendix II of the Convention on International Trade in Endangered Species provides t...,loss;,,Endangered,"['Taiwan', 'China', 'Vietnam']",3,1,1.0
25,4646125,"malagasy giant jumping rat, malagasy giant rat",Nesomyidae,Terrestrial,Madagascar;,\r\r\r\r\r\n The historical decline of this species has been partly through climatic change l...,['\n The new Menabe-Antimena protected area has temporary protection order and covers the ent...,Decreasing,Endangered,1,...,1 Ã¢?? 1.5 kg (2),Like many of MadagascarÃ¢??s unique species the Malagasy giant rat is thought to have become hig...,This large rodent is in urgent need of conservation and its future remains highly uncertain. The...,loss;pet;,captive breeding;protected;,Endangered,['Canada'],1,2,1.0
33,8966650,lydenburg cycad,Zamiaceae,Terrestrial,South Africa (Limpopo Province);,\r\r\r\r\r\n This species has suffered much from the activities of collectors and in addition...,['\n This species is listed on Appendix I of the CITES Appendices.\n\n \n '],Decreasing,Critically Endangered,1,...,Height: up to 3 m (2),Over the past few decades many South African cycads have become increasingly scarce in the wild ...,There are not known to be any specific conservation measures in place for this Critically Endang...,loss;,cites;protected;,Critically Endangered,"['Australia', 'South Africa', 'Brazil']",3,1,1.0
48,1199442,"bluelegged mantella, tular golden frog, tular mantella, tulear golden frog",Mantellidae,Terrestrial; Freshwater,Madagascar;,"\r\r\r\r\r\n The main threat to this species is habitat loss due to grazing and fire, and in ...",['\n It occurs in Parque Nacional de Isalo. Trade in this s

In [3]:
for c in t.columns:
    print(c)

_id
ltable_name
ltable_family
ltable_ecology
ltable_countries
ltable_threat_paragraph
ltable_conservation_paragraph
ltable_pop_trend
ltable_status
ltable_country_count
scientific_name
rtable_name
rtable_kingdom
rtable_phylum
rtable_class
rtable_order
rtable_family
genus
rtable_size
rtable_threats
rtable_conservation
rtable_threat_keywords
rtable_conservation_keywords
rtable_status
rtable_countries
rtable_country_count
rtable_tCount
label


In [ ]:
t.head()

In [6]:
print('Number of tuples:', len(t))
# print('Number of unique rtable values', len(t.rtable_scientific_name.unique()))
print('Number of unique genuses:', len(t.genus.unique()))
print('Number of unique sizes:', len(t.rtable_size.unique()))

Number of tuples: 148
Number of unique genuses: 92
Number of unique sizes: 102


In [13]:
for u in t.rtable_size.unique():
    print(u)
    print()

Extended length: up to 90 mm (2)

Height: 3 - 7 m (2)Trunk diameter: c. 20 cm (2)

700 Ã¢?? 900 g (2)

1 Ã¢?? 3 g (3)

1 Ã¢?? 1.5 kg (2)

Height: up to 3 m (2)

Snout-vent length: up to 8 cm (2)

5.8 Ã¢?? 7.4 kg (2)

Length: 6 - 8 cm (2)

c. 10 g (2)

Snout-vent length: 51 - 68 mm (2) (3)Tail length: c. 51 mm (2)

Male head-body length: c. 42.7 cm (2)Female head-body length: c. 41.8 cm (2)Tail length: 30 - 50.7 cm (3)

Length: up to 7.6 cm (2)

Length: 6 mm (2).

Trunk diameter: up to 172 cm (2)

up to 10 g (2)

ca. 7 kg (2)

1 - 1.5 kg (2)

Wingspan: 19 Ã¢?? 28 cm (2)

up to 53 kg (2)

Male head-to-body length: 57 Ã¢?? 62 cm (2)Female head-to-body length: 57 Ã¢?? 59 cm (2)Mail tail length: 82 - 88 cm (2)Female tail length: 84 Ã¢?? 86 cm (2)Male weight: 7.5 Ã¢?? 10.5 kg (2)Female weight: 6.2 Ã¢?? 9.2 kg (2)

Maximum snout-vent length: 92 cm (2)

Male length: c. 71 mm (2)Female length: c. 74 mm (2)

Plant size: 1 - 5 mm

2.5 kg (2)

Length: 15 Ã¢?? 18 mm (2)

2.5 - 4 kg (4)

Megophrys (

In [35]:
# Figure out how to parse out weight from size column
u = t.rtable_size.unique()
lengths = [l for l in u if isinstance(l, str) and 'length' in l]
print('number of lengths:', len(lengths))
notlengths = [l for l in u if isinstance(l, str) and 'length' not in l]
weights = [l for l in u if isinstance(l, str) and (' g ' in l or ' kg ' in l)]
print('Number of weights', len(weights))
print()
print()
print()
for w in weights:
    
    match = None
    # find last occurrance of "<number> g" or "<number> kg"
    for match in re.finditer(r"\d+ (k)?g", w):
        pass
    #only keep the digit
    m = float(re.sub(r'\D', "", match.group()))
    if 'k' in match.group():
        m *= 1000
    print(match.group())
    print(m)
    print()

for size in t.rtable_size.unique():
    print(size)
    print()


number of lengths: 30
Number of weights 33



900 g
900.0

3 g
3.0

5 kg
5000.0

4 kg
4000.0

10 g
10.0

10 g
10.0

7 kg
7000.0

5 kg
5000.0

53 kg
53000.0

2 kg
2000.0

5 kg
5000.0

4 kg
4000.0

315 kg
315000.0

179 g
179.0

320 kg
320000.0

115 g
115.0

4 g
4.0

88 g
88.0

4 kg
4000.0

3 kg
3000.0

6 kg
6000.0

180 g
180.0

5 kg
5000.0

130 kg
130000.0

60 g
60.0

5 kg
5000.0

8 kg
8000.0

40 kg
40000.0

25 kg
25000.0

60 g
60.0

5 kg
5000.0

34 g
34.0

1000 g
1000.0

Extended length: up to 90 mm (2)

Height: 3 - 7 m (2)Trunk diameter: c. 20 cm (2)

700 Ã¢?? 900 g (2)

1 Ã¢?? 3 g (3)

1 Ã¢?? 1.5 kg (2)

Height: up to 3 m (2)

Snout-vent length: up to 8 cm (2)

5.8 Ã¢?? 7.4 kg (2)

Length: 6 - 8 cm (2)

c. 10 g (2)

Snout-vent length: 51 - 68 mm (2) (3)Tail length: c. 51 mm (2)

Male head-body length: c. 42.7 cm (2)Female head-body length: c. 41.8 cm (2)Tail length: 30 - 50.7 cm (3)

Length: up to 7.6 cm (2)

Length: 6 mm (2).

Trunk diameter: up to 172 cm (2)

up to 10 g (2)

ca. 7 k

In [11]:
#add weight column


weightColumn = []

# replace name fields with list of nick/common names
for index, row in t.iterrows():
    size = row['rtable_size']
    if not size:
        continue
    #if this is a weight
    if isinstance(size, str) and (' g ' in size or ' kg ' in size):
        match = None
        # find last occurrance of "<number> g" or "<number> kg"
        for match in re.finditer(r"\d (k)?g", size):
            pass
        #only keep the digit
        m = float(re.sub(r'\D', "", match.group()))
        if 'km' in match.group():
            m *= 1000
        weightColumn.append(m)
        
    else:
        weightColumn.append(None)
        
t['weight'] = weightColumn
t.head()

,_id,ltable_name,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,ltable_status,ltable_country_count,...,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_status,rtable_countries,rtable_country_count,rtable_tCount,label,weight
7,1522478,caterpillar slug,Veronicellidae,NaN,South Africa (KwaZulu-Natal);,NaN,NaN,Unknown,Endangered,1,...,The caterpillar slug is threatened by habitat loss and degradation as a result of ongoing urbani...,Although there are currently no conservation measures directly targeting the caterpillar slug in...,loss;environment;,,Endangered,"['India', 'Russia', 'Malaysia', 'China', 'Indonesia']",5,2,1.0,NaN
17,8689373,catalina mahogany,Rosaceae,NaN,United States (California);,NaN,NaN,NaN,Critically Endangered,1,...,Historically a major threat to the Catalina mahogany was the introduction of <strong>herbivores<...,Conservation efforts began in the 1970s with a detailed inventory of the remaining Catalina maho...,loss;invasive;,,Critically Endangered,"['Ukraine', 'Morocco', 'Russia', 'Hungary']",4,2,1.0,NaN
21,7512846,"lorenz von liburnaus woolly lemur, western avahi, western woolly lemur",Indriidae,Terrestrial,Madagascar;,\r\r\r\r\r\n The major threat is forest destruction due to annual burning that creates new ca...,"['\n This species is listed on Appendix I of CITES. ', <span lang=""EN-CA"">This species is kno...",Decreasing,Endangered,1,...,15 species of lemur have become extinct since sea-faring humans arrived on MadagascarÃ¢??s shore...,The western woolly lemur is confirmed in only two protected areas Ankarafantsika Nature Reserve ...,hunting;,protected;,Critically Endangered,['Australia'],1,1,1.0,0.0
23,1191527,"bluelegged mantella, tular golden frog, tular mantella, tulear golden frog",Mantellidae,Terrestrial; Freshwater,Madagascar;,"\r\r\r\r\r\n The main threat to this species is habitat loss due to grazing and fire, and in ...",['\n It occurs in Parque Nacional de Isalo. Trade in this species needs to be very carefully ...,Decreasing,Endangered,1,...,Several thousand blue-legged mantellas are thought to be collected every year from some regions ...,Listing on Appendix II of the Convention on International Trade in Endangered Species provides t...,loss;,,Endangered,"['Taiwan', 'China', 'Vietnam']",3,1,1.0,3.0
25,4646125,"malagasy giant jumping rat, malagasy giant rat",Nesomyidae,Terrestrial,Madagascar;,\r\r\r\r\r\n The historical decline of this species has been partly through climatic change l...,['\n The new Menabe-Antimena protected area has temporary protection order and covers the ent...,Decreasing,Endangered,1,...,Like many of MadagascarÃ¢??s unique species the Malagasy giant rat is thought to have become hig...,This large rodent is in urgent need of conservation and its future remains highly uncertain. The...,loss;pet;,captive breeding;protected;,Endangered,['Canada'],1,2,1.0,5000.0


In [50]:
#Figure out how to parse length

def strToMeters(srcString):
    n = float(re.sub(r'[^0-9.]', "", srcString))
    if 'cm' in srcString:
        n /= 100
    if 'mm' in srcString:
        n /= 1000
    return n
    
u = t.rtable_size.unique()
lengths = [l for l in u if isinstance(l, str) and 'length' in l]
print('number of lengths:', len(lengths))
out = ''
numMatches = 0;
for l in lengths:
    out += '\n'
    out += l + '\n'
    match = None
    # find last occurrance of "<number> cm" 
    # or "<number> mm" or "<number m"
    maxLen = 0
    maxStr = ""
    for match in re.finditer(r"([0-9]+.)?[0-9]+ (c)?(m+)", l, re.I):
        tmp = strToMeters(match.group())
        if maxLen < tmp:
            maxLen = tmp
            maxStr = match.group()
            
    #only keep the digit
    if not match:
        continue
    numMatches += 1
    
    out += maxStr + '\n' + str(maxLen) + '\n'
    
print('number of matches:', numMatches)
print()
print()
print()
print(out)


number of lengths: 30
number of matches: 30




Extended length: up to 90 mm (2)
90 mm
0.09

Snout-vent length: up to 8 cm (2)
8 cm
0.08

Snout-vent length: 51 - 68 mm (2) (3)Tail length: c. 51 mm (2)
68 mm
0.068

Male head-body length: c. 42.7 cm (2)Female head-body length: c. 41.8 cm (2)Tail length: 30 - 50.7 cm (3)
50.7 cm
0.507

Male head-to-body length: 57 Ã¢?? 62 cm (2)Female head-to-body length: 57 Ã¢?? 59 cm (2)Mail tail length: 82 - 88 cm (2)Female tail length: 84 Ã¢?? 86 cm (2)Male weight: 7.5 Ã¢?? 10.5 kg (2)Female weight: 6.2 Ã¢?? 9.2 kg (2)
88 cm
0.88

Maximum snout-vent length: 92 cm (2)
92 cm
0.92

Male length: c. 71 mm (2)Female length: c. 74 mm (2)
74 mm
0.074

Male length: 20 Ã¢?? 30 mm (2)Female length: 30 Ã¢?? 40 mm (2)
40 mm
0.04

Height: 90 Ã¢?? 180 cm (2)Leaf length: 7.5 - 15 cm (2)Flower length: up to 7.5 cm (2)
180 cm
1.8

Male length: 68 - 78 mm (2)Female length: 78-83 mm (2)Male length of abdomen: 50 - 60 mm (2)Female length of abdomen: 59 - 63 mm (2)Male hin

In [51]:
def strToMeters(srcString):
    n = float(re.sub(r'[^0-9.]', "", srcString))
    if 'cm' in srcString:
        n /= 100
    if 'mm' in srcString:
        n /= 1000
    return n

#add length column

lengthColumn = []

# replace name fields with list of nick/common names
for index, row in t.iterrows():
    size = row['rtable_size']
    if not size:
        continue
    #if this is a length
    if isinstance(size, str) and 'length' in size:
        match = None
        maxLen = 0
        maxStr = ""
        # find last occurrance of "<number> g" or "<number> kg"
        for match in re.finditer(r"([0-9]+.)?[0-9]+ (c)?(m+)", 
                                 size, re.I):
            tmp = strToMeters(match.group())
            if maxLen < tmp:
                maxLen = tmp
                maxStr = match.group()
        
        lengthColumn.append(maxLen)
        
    else:
        lengthColumn.append(None)

t['length'] = lengthColumn
t.head()

,_id,ltable_name,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,ltable_status,ltable_country_count,...,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_status,rtable_countries,rtable_country_count,rtable_tCount,label,weight,length
7,1522478,caterpillar slug,Veronicellidae,NaN,South Africa (KwaZulu-Natal);,NaN,NaN,Unknown,Endangered,1,...,Although there are currently no conservation measures directly targeting the caterpillar slug in...,loss;environment;,,Endangered,"['India', 'Russia', 'Malaysia', 'China', 'Indonesia']",5,2,1.0,NaN,0.09
17,8689373,catalina mahogany,Rosaceae,NaN,United States (California);,NaN,NaN,NaN,Critically Endangered,1,...,Conservation efforts began in the 1970s with a detailed inventory of the remaining Catalina maho...,loss;invasive;,,Critically Endangered,"['Ukraine', 'Morocco', 'Russia', 'Hungary']",4,2,1.0,NaN,NaN
21,7512846,"lorenz von liburnaus woolly lemur, western avahi, western woolly lemur",Indriidae,Terrestrial,Madagascar;,\r\r\r\r\r\n The major threat is forest destruction due to annual burning that creates new ca...,"['\n This species is listed on Appendix I of CITES. ', <span lang=""EN-CA"">This species is kno...",Decreasing,Endangered,1,...,The western woolly lemur is confirmed in only two protected areas Ankarafantsika Nature Reserve ...,hunting;,protected;,Critically Endangered,['Australia'],1,1,1.0,0.0,NaN
23,1191527,"bluelegged mantella, tular golden frog, tular mantella, tulear golden frog",Mantellidae,Terrestrial; Freshwater,Madagascar;,"\r\r\r\r\r\n The main threat to this species is habitat loss due to grazing and fire, and in ...",['\n It occurs in Parque Nacional de Isalo. Trade in this species needs to be very carefully ...,Decreasing,Endangered,1,...,Listing on Appendix II of the Convention on International Trade in Endangered Species provides t...,loss;,,Endangered,"['Taiwan', 'China', 'Vietnam']",3,1,1.0,3.0,NaN
25,4646125,"malagasy giant jumping rat, malagasy giant rat",Nesomyidae,Terrestrial,Madagascar;,\r\r\r\r\r\n The historical decline of this species has been partly through climatic change l...,['\n The new Menabe-Antimena protected area has temporary protection order and covers the ent...,Decreasing,Endangered,1,...,This large rodent is in urgent need of conservation and its future remains highly uncertain. The...,loss;pet;,captive breeding;protected;,Endangered,['Canada'],1,2,1.0,5000.0,NaN


In [52]:
t.to_csv('labeled.csv')